In [1]:
# Qpt

In [2]:
import pickle
import time
from typing import List
import datetime as dt
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

from tqdm import tqdm

import numpy.testing as npt

from quara.data_analysis import data_analysis, physicality_violation_check, report
from quara.data_analysis.projected_gradient_descent_backtracking import ProjectedGradientDescentBacktracking, ProjectedGradientDescentBacktrackingOption
from quara.data_analysis.weighted_probability_based_squared_error import WeightedProbabilityBasedSquaredError, WeightedProbabilityBasedSquaredErrorOption
from quara.data_analysis.weighted_relative_entropy import WeightedRelativeEntropy, WeightedRelativeEntropyOption
from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis
from quara.objects.state import (
    State,
    get_x0_1q,
    get_x1_1q,
    get_y0_1q,
    get_y1_1q,
    get_z0_1q,
    get_z1_1q,
)
from quara.objects.povm import (
    Povm,get_x_measurement,
    get_y_measurement,
    get_z_measurement
)
from quara.objects.gate import (
    Gate, get_depolarizing_channel,
    get_x_rotation,
    get_amplitutde_damping_channel
)
from quara.objects.qoperation import QOperation
from quara.protocol.qtomography.standard.standard_qpt import StandardQpt
from quara.protocol.qtomography.standard.linear_estimator import LinearEstimator
from quara.protocol.qtomography.standard.loss_minimization_estimator import LossMinimizationEstimator
from quara.protocol.qtomography.standard.projected_linear_estimator import (
    ProjectedLinearEstimator,
)

from quara.data_analysis.simulation import SimulationSetting

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
start_all = time.time()

In [5]:
# setup system
e_sys = ElementalSystem(0, get_normalized_pauli_basis())
c_sys = CompositeSystem([e_sys])

# Tester Objects (State)
# |+><+|
state_x0 = get_x0_1q(c_sys)
# |+i><+i|
state_y0 = get_y0_1q(c_sys)
# |0><0|
state_z0 = get_z0_1q(c_sys)
# |1><1|
state_z1 = get_z1_1q(c_sys)
tester_states = [state_x0, state_y0, state_z0, state_z1]

In [6]:
tester_povms = [get_x_measurement(c_sys), get_y_measurement(c_sys), get_z_measurement(c_sys)]

In [7]:
# True Object
true_objects = []
#true_objects.append(get_depolarizing_channel(p=0, c_sys=c_sys))
true_objects.append(get_depolarizing_channel(p=0.05, c_sys=c_sys))
#true_objects.append(get_depolarizing_channel(p=1, c_sys=c_sys))
#true_objects.append(get_x_rotation(theta=np.pi/2, c_sys=c_sys))
#true_objects.append(get_x_rotation(theta=np.pi, c_sys=c_sys))
#true_objects.append(get_amplitutde_damping_channel(gamma=0.1, c_sys=c_sys))

true_object = true_objects[0]

In [8]:
num_data = [100, 1000, 10000]
n_rep = 10

case_name_list = [
    "LinearEstimator(True)",
    "LinearEstimator(False)",
    "ProjectedLinearEstimator(True)",
    "ProjectedLinearEstimator(False)",
    "Maximum-Likelihood(True)",
    "Maximum-Likelihood(False)",
    "Least Squares(True)",
    "Least Squares(False)",
]

seed = 777
qtomography_list = [
    StandardQpt(tester_states, tester_povms, on_para_eq_constraint=True, seed=seed),
    StandardQpt(tester_states, tester_povms, on_para_eq_constraint=False, seed=seed),
    StandardQpt(tester_states, tester_povms, on_para_eq_constraint=True, seed=seed),
    StandardQpt(tester_states, tester_povms, on_para_eq_constraint=False, seed=seed),
    StandardQpt(tester_states, tester_povms, on_para_eq_constraint=True, eps_proj_physical=1e-13, seed=seed),
    StandardQpt(tester_states, tester_povms, on_para_eq_constraint=False, eps_proj_physical=1e-13, seed=seed),
    StandardQpt(tester_states, tester_povms, on_para_eq_constraint=True, eps_proj_physical=1e-13, seed=seed),
    StandardQpt(tester_states, tester_povms, on_para_eq_constraint=False, eps_proj_physical=1e-13, seed=seed),
]

para_list = [
    True,
    False,
    True,
    False,
    True,
    False,
    True,
    False,
]

estimator_list = [
    LinearEstimator(),
    LinearEstimator(),
    ProjectedLinearEstimator(),
    ProjectedLinearEstimator(),
    LossMinimizationEstimator(),
    LossMinimizationEstimator(),
    LossMinimizationEstimator(),
    LossMinimizationEstimator(),
]

loss_list = [
    None,
    None,
    None,
    None,
    WeightedRelativeEntropy(12),
    WeightedRelativeEntropy(16),
    WeightedProbabilityBasedSquaredError(12),
    WeightedProbabilityBasedSquaredError(16),
]

loss_option_list = [
    None,
    None,
    None,
    None,
    WeightedRelativeEntropyOption(),
    WeightedRelativeEntropyOption(),
    WeightedProbabilityBasedSquaredErrorOption(),
    WeightedProbabilityBasedSquaredErrorOption(),
]

algo_list = [
    None,
    None,
    None,
    None,
    ProjectedGradientDescentBacktracking(),
    ProjectedGradientDescentBacktracking(),
    ProjectedGradientDescentBacktracking(),
    ProjectedGradientDescentBacktracking(),
]

algo_option_list = [
    None,
    None,
    None,
    None,
    ProjectedGradientDescentBacktrackingOption(),
    ProjectedGradientDescentBacktrackingOption(),
    ProjectedGradientDescentBacktrackingOption(),
    ProjectedGradientDescentBacktrackingOption(),
]

estimation_results_list = []
elapsed_times = []
simulation_settings = []

for i, name in enumerate(case_name_list):
    qtomography = qtomography_list[i]
    estimator = estimator_list[i]
    loss = loss_list[i]
    loss_option = loss_option_list[i]
    algo = algo_list[i]
    algo_option = algo_option_list[i]

    start = time.time()
    print(f"Case {i}: {name}")
    print(f"Parametorization: {para_list[i]}")
    print(f"Type of qtomography: {qtomography.__class__.__name__}")
    print(f"Estimator: {estimator.__class__.__name__}")

    estimation_results = data_analysis.estimate(
       qtomography=qtomography,
       true_object=true_object,
       num_data=num_data,
       estimator=estimator,
       loss=loss,
       loss_option=loss_option,
       algo=algo,
       algo_option=algo_option,
       iteration=n_rep,
    )
    estimation_results_list.append(estimation_results)
    
    # stock settings of this simulation
    simulation_setting = SimulationSetting(
        name=name,
        estimator=estimator,
        loss=loss,
        loss_option=loss_option,
        algo=algo,
        algo_option=algo_option,
    )
    simulation_settings.append(simulation_setting)

    elapsed_time = time.time() - start
    print("elapsed_time:{0}".format(elapsed_time / 60) + "[min]\n")
    elapsed_times.append(elapsed_time)

  0%|          | 0/10 [00:00<?, ?it/s]Case 0: LinearEstimator(True)
Parametorization: True
Type of qtomography: StandardQpt
Estimator: LinearEstimator
  0%|          | 0/10 [00:00<?, ?it/s]elapsed_time:0.04635001420974731[min]

Case 1: LinearEstimator(False)
Parametorization: False
Type of qtomography: StandardQpt
Estimator: LinearEstimator
  0%|          | 0/10 [00:00<?, ?it/s]elapsed_time:0.05019368728001913[min]

Case 2: ProjectedLinearEstimator(True)
Parametorization: True
Type of qtomography: StandardQpt
Estimator: ProjectedLinearEstimator
  0%|          | 0/10 [00:00<?, ?it/s]elapsed_time:0.07250612179438273[min]

Case 3: ProjectedLinearEstimator(False)
Parametorization: False
Type of qtomography: StandardQpt
Estimator: ProjectedLinearEstimator
  0%|          | 0/10 [00:00<?, ?it/s]elapsed_time:0.07599856853485107[min]

Case 4: Maximum-Likelihood(True)
Parametorization: True
Type of qtomography: StandardQpt
Estimator: LossMinimizationEstimator
  0%|          | 0/10 [00:00<?, ?it/

In [9]:
fig = data_analysis.make_mses_graph_estimation_results(
    estimation_results_list,
    case_name_list,
    true_object,
)
fig.show()

  LinearEstimator(True)=[0.12242000000000011, 0.007828800000000014, 0.0011080960000000057]
  LinearEstimator(False)=[0.10006000000000004, 0.0080212, 0.0007524879999999958]
  ProjectedLinearEstimator(True)=[0.03971116167556233, 0.005768569599730462, 0.0008710045046660692]
  ProjectedLinearEstimator(False)=[0.04168500257516522, 0.005627817030519908, 0.0008578890167040079]
  Maximum-Likelihood(True)=[0.024885475655293005, 0.0060567919208944125, 0.0007481036130290633]
  Maximum-Likelihood(False)=[0.020068118883699124, 0.004708859888961943, 0.0007895989873904962]
  Least Squares(True)=[0.04328354122136216, 0.005691538724860264, 0.0006641046520211313]
  Least Squares(False)=[0.030827762172485236, 0.003998602715766462, 0.0009840978374772112]


In [10]:
report.export_report(
    "qpt_nrep=10.pdf",
    estimation_results_list=estimation_results_list,
    simulation_settings=simulation_settings,
    true_object=true_object,
    tester_objects=tester_states+tester_povms,
    seed=seed,
    computation_time=sum(elapsed_times)
)

y.standard.linear_estimator.LinearEstimationResult object at 0x0000024CCC3FC908>], [<quara.protocol.qtomography.standard.projected_linear_estimator.ProjectedLinearEstimationResult object at 0x0000024CCC3FCB00>, <quara.protocol.qtomography.standard.projected_linear_estimator.ProjectedLinearEstimationResult object at 0x0000024CCC3FCB38>, <quara.protocol.qtomography.standard.projected_linear_estimator.ProjectedLinearEstimationResult object at 0x0000024CCC42FE48>, <quara.protocol.qtomography.standard.projected_linear_estimator.ProjectedLinearEstimationResult object at 0x0000024CCC42F518>, <quara.protocol.qtomography.standard.projected_linear_estimator.ProjectedLinearEstimationResult object at 0x0000024CCC42FA58>, <quara.protocol.qtomography.standard.projected_linear_estimator.ProjectedLinearEstimationResult object at 0x0000024CCC42F3C8>, <quara.protocol.qtomography.standard.projected_linear_estimator.ProjectedLinearEstimationResult object at 0x0000024CCC42F390>, <quara.protocol.qtomography